# Running ALS on MovieLens (pySpark)

ALS (Alternating Least Squares) is a well-known collaborative filtering algorithm.

This notebook provides an example of how to utilize and evaluate ALS pySpark ML implementation, meant for large-scale distributed datasets. We use a smaller dataset in this example to run SAR efficiently on Data Science Virtual Machine.

In [15]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")
import os
import numpy as np
from zipfile import ZipFile
import papermill as pm
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType

from reco_utils.dataset.url_utils import maybe_download
from reco_utils.dataset.spark_splitters import spark_random_split
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation


print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))


System version: 3.6.0 | packaged by conda-forge | (default, Feb  9 2017, 14:36:55) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
Spark version: 2.3.1


Set the default parameters.

In [16]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### 0. Set up Spark context

The following settings work well for debugging locally on VM - change when running on a cluster. We set up a giant single executor with many threads and specify memory cap. 

In [17]:
# the following settings work well for debugging locally on VM - change when running on a cluster
# set up a giant single executor with many threads and specify memory cap
spark = SparkSession \
    .builder \
    .appName("SAR pySpark") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g")\
    .config("spark.executor.cores", "32")\
    .config("spark.executor.memory", "8g")\
    .config("spark.memory.fraction", "0.9")\
    .config("spark.memory.stageFraction", "0.3")\
    .config("spark.executor.instances", 1)\
    .config("spark.executor.heartbeatInterval", "36000s")\
    .config("spark.network.timeout", "10000000s")\
    .config("spark.driver.maxResultSize", "50g")\
    .getOrCreate()


### 1. Download the MovieLens dataset

In [19]:
# MovieLens data have different data-format for each size of dataset
data_header = False
if MOVIELENS_DATA_SIZE == "100k":
    separator = "\t"
    data_name = "u.data"
    data_folder = "ml-100k"
elif MOVIELENS_DATA_SIZE == "1m":
    separator = "::"
    data_name = "ratings.dat"
    data_folder = "ml-1m"
elif MOVIELENS_DATA_SIZE == "10m":
    separator = "::"
    data_name = "ratings.dat"
    data_folder = "ml-10M100K"
elif MOVIELENS_DATA_SIZE == "20m":
    separator = ","
    data_name = "ratings.csv"
    data_folder = "ml-20m"
    data_header = True
else:
    raise ValueError("Invalid data size. Should be one of {100k, 1m, 10m, or 20m}")

# Download dataset zip file and decompress if haven't done yet
data_path = os.path.join(data_folder, data_name)
if not os.path.exists(data_path):
    filename = "ml-" + MOVIELENS_DATA_SIZE + ".zip"
    filepath = maybe_download(
        "http://files.grouplens.org/datasets/movielens/" + filename, filename
    )

    with ZipFile(filepath, "r") as zf:
        zf.extractall()

    # remove zip file we already used
    os.remove(filepath)


In [20]:
schema = StructType(
    (
        StructField("UserId", IntegerType()),
        StructField("MovieId", IntegerType()),
        StructField("Rating", FloatType()),
        StructField("Timestamp", IntegerType()),
    )
)

# pySpark's read csv currently doesn't support multi-character delimiter, thus we manually handle that
if len(separator) > 1:
    raw_data = spark.sparkContext.textFile(data_path) \
        .map(lambda l: l.split(separator)).map(lambda s: ','.join(map(str, s)))
    data = spark.read.csv(raw_data, schema=schema, sep=',', header=data_header)
else:
    data = spark.read.csv(data_path, schema=schema, sep=separator, header=data_header)

data.show()

+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
|   244|     51|   2.0|880606923|
|   166|    346|   1.0|886397596|
|   298|    474|   4.0|884182806|
|   115|    265|   2.0|881171488|
|   253|    465|   5.0|891628467|
|   305|    451|   3.0|886324817|
|     6|     86|   3.0|883603013|
|    62|    257|   2.0|879372434|
|   286|   1014|   5.0|879781125|
|   200|    222|   5.0|876042340|
|   210|     40|   3.0|891035994|
|   224|     29|   3.0|888104457|
|   303|    785|   3.0|879485318|
|   122|    387|   5.0|879270459|
|   194|    274|   2.0|879539794|
|   291|   1042|   4.0|874834944|
|   234|   1184|   2.0|892079237|
+------+-------+------+---------+
only showing top 20 rows



### 2. Split the data using the Spark random splitter provided in utilities

In [21]:
train, test = spark_random_split(data, ratio=0.75, seed=123)
print ("N train", train.count())
print ("N test", test.count())

N train 75193
N test 24807


In [22]:
header = {
    "userCol": "UserId",
    "itemCol": "MovieId",
    "ratingCol": "Rating",
}

# Even we uses explicit rating, implicitPrefs=True produces higher accuracy
als = ALS(
    rank=40,
    maxIter=15,
    implicitPrefs=True,
    alpha=0.1,
    coldStartStrategy='drop',
    **header
)

### 3. Train the ALS model on the training data, and get the top-k recommendations for our testing data

In [23]:
model = als.fit(train)
recommendations = model.recommendForUserSubset(test, TOP_K)

In [24]:
recommendations.show()

+------+--------------------+
|UserId|     recommendations|
+------+--------------------+
|   471|[[501, 0.51323473...|
|   463|[[285, 0.69717616...|
|   833|[[56, 0.7414067],...|
|   496|[[173, 0.54708683...|
|   148|[[408, 0.522487],...|
|   540|[[1, 0.67718685],...|
|   392|[[286, 0.6472844]...|
|   243|[[275, 0.57424104...|
|   623|[[50, 0.53407395]...|
|   737|[[127, 0.35307357...|
|   897|[[204, 0.795846],...|
|   858|[[286, 0.48657185...|
|    31|[[303, 0.54063654...|
|   516|[[286, 0.25444984...|
|   580|[[405, 0.6270535]...|
|   251|[[181, 0.8744547]...|
|   451|[[327, 0.7952845]...|
|    85|[[213, 0.77625227...|
|   137|[[121, 0.7106164]...|
|   808|[[313, 0.6403519]...|
+------+--------------------+
only showing top 20 rows



In [25]:
# Convert to reco util's ranking evaluator format
top_k = recommendations.select('UserId', F.explode('recommendations').alias('r')) \
    .select('UserId', 'r.*')
top_k.show()

+------+-------+----------+
|UserId|MovieId|    rating|
+------+-------+----------+
|   471|    501|0.51323473|
|   471|    418| 0.5080572|
|   471|    946|0.46807322|
|   471|    596| 0.4133107|
|   471|     99| 0.4084396|
|   471|    102| 0.3998096|
|   471|    420|0.39486644|
|   471|    588| 0.3918579|
|   471|    465|0.38982785|
|   471|    969|0.38271752|
|   463|    285|0.69717616|
|   463|    475| 0.6278335|
|   463|    124|0.62751156|
|   463|    286| 0.6239718|
|   463|     13|0.61030906|
|   463|    116| 0.5977628|
|   463|    100|0.59301853|
|   463|     14| 0.5878561|
|   463|    302|0.58484685|
|   463|    258| 0.5802202|
+------+-------+----------+
only showing top 20 rows



### 4. Evaluate how well SAR performs 

In [26]:
test.show()

+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|     1|      2|   3.0|876893171|
|     1|      3|   4.0|878542960|
|     1|      4|   3.0|876893119|
|     1|      9|   5.0|878543541|
|     1|     11|   2.0|875072262|
|     1|     17|   3.0|875073198|
|     1|     25|   4.0|875071805|
|     1|     28|   4.0|875072173|
|     1|     30|   3.0|878542515|
|     1|     33|   4.0|878542699|
|     1|     43|   4.0|878542869|
|     1|     48|   5.0|875072520|
|     1|     49|   3.0|878542478|
|     1|     52|   4.0|875072205|
|     1|     59|   5.0|876892817|
|     1|     62|   3.0|878542282|
|     1|     65|   4.0|875072125|
|     1|     66|   4.0|878543030|
|     1|     71|   3.0|876892425|
|     1|     78|   1.0|878543176|
+------+-------+------+---------+
only showing top 20 rows



In [27]:
rank_eval = SparkRankingEvaluation(test, top_k, k = TOP_K, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="rating", 
                                    relevancy_method="top_k")

In [28]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.026486
NDCG:	0.108293
Precision@K:	0.104352
Recall@K:	0.080269


In [ ]:
# Record results with papermill for tests
pm.record("map", rank_eval.map_at_k())
pm.record("ndcg", rank_eval.ndcg_at_k())
pm.record("precision", rank_eval.precision_at_k())
pm.record("recall", rank_eval.recall_at_k())
